# Trabajo Práctico N° 2: Redes Neuronales
## Natalia Mellino - Introducción al Aprendizaje Automatizado

In [1]:
# imports here

import numpy as np
import sklearn as sk
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from copy import deepcopy
import matplotlib.pyplot as plt
import espirales_anidadas

In [2]:
#defino parámetros de la red
epocas_por_entrenamiento=25    #numero de epocas que entrena cada vez
eta=0.01              #learning rate
alfa=0.9               #momentum
N2=60                 #neuronas en la capa oculta

#defino MLP para regresión
regr = MLPRegressor(hidden_layer_sizes=(N2,), activation='logistic', solver='sgd', alpha=0.0, batch_size=1, learning_rate='constant', learning_rate_init=eta,momentum=alfa,nesterovs_momentum=False,tol=0.0,warm_start=True,max_iter=epocas_por_entrenamiento)
#defino MLP para clasificación
clasif = MLPClassifier(hidden_layer_sizes=(N2,), activation='logistic', solver='sgd', alpha=0.0, batch_size=1, learning_rate='constant', learning_rate_init=eta,momentum=alfa,nesterovs_momentum=False,tol=0.0,warm_start=True,max_iter=epocas_por_entrenamiento)
print(regr)

MLPRegressor(activation='logistic', alpha=0.0, batch_size=1,
             hidden_layer_sizes=(60,), learning_rate_init=0.01, max_iter=25,
             nesterovs_momentum=False, solver='sgd', tol=0.0, warm_start=True)


In [3]:
# Entrenar red para REGRESION

def entrenar_red_regresion(red, evaluaciones, X_train, y_train, X_val, y_val, X_test, y_test):
    best_red = None
    error_train = None
    error_val = None
    error_test = None
    for i in range(evaluaciones):
        # Entrenamos
        model = red.fit(X_train, y_train)
        # Predecimos
        pred_train = model.predict(X_train)
        pred_val = model.predict(X_val)
        pred_test = model.predict(X_test)
        # Guardamos error y vemos si la red que tenemos es mejor que la anterior
        current_error_train = sk.metrics.mean_squared_error(y_train, pred_train)
        current_error_val = sk.metrics.mean_squared_error(y_val, pred_val)
        current_error_test = sk.metrics.mean_squared_error(y_test, pred_test)

        if error_val is None or current_error_val < error_val:
            best_red = deepcopy(model)
            error_train = current_error_train
            error_test = current_error_test
            error_val = current_error_val

    return best_red, error_train, error_val, error_test

In [5]:
# Entrenar red para CLASIFICACIÓN

def entrenar_red_clasificacion(red, evaluaciones, X_train, y_train, X_val, y_val, X_test, y_test):
    best_red = None
    error_train = None
    error_val = None
    error_test = None
    for i in range(evaluaciones):
        # Entrenamos
        model = red.fit(X_train, y_train)
        # Predecimos
        pred_train = model.predict(X_train)
        pred_val = model.predict(X_val)
        pred_test = model.predict(X_test)
        # Guardamos error y vemos si la red que tenemos es mejor que la anterior
        current_error_train = sk.metrics.zero_one_loss(y_train, pred_train)
        current_error_val = sk.metrics.zero_one_loss(y_val, pred_val)
        current_error_test = sk.metrics.zero_one_loss(y_test, pred_test)

        if error_val is None or current_error_val < error_val:
            best_red = deepcopy(model)
            error_train = current_error_train
            error_test = current_error_test
            error_val = current_error_val

    return best_red, error_train, error_val, error_test

In [ ]:
# Función para graficar errores y predicciones

# regr, e_train, e_val, e_test = entrenar_red_regresion(regr, epocas, X_train, y_train, X_val, y_val, X_test, y_test)

def graficar_errores(e_train, e_val, e_test, epocas):
    plt.plot(range(epocas), e_train, label="train", linestyle=":")
    plt.plot(range(epocas), e_val, label="validacion", linestyle="-.")
    plt.plot(range(epocas), e_test, label="test", linestyle="-")
    plt.legend()
    plt.show()

## Ejercicio 1

In [ ]:
def ej1():
    train = espirales_anidadas.generate_points_c(600)
    test = espirales_anidadas.generate_points_c(2000)

    evaluaciones = 1000
    epocas = 20
    eta = 0.1 # lr
    alfa = 0.9 # m

    n_neuronas = [2, 10, 20, 40]